# Example: Shallow ReLU Network ROM with ONERA M6 Data

NN = Neural Network

In [1]:
import rom
import numpy as np

The ONERA M6 drag data can be found [here](https://github.com/kaylabollinger/ROM/tree/main/data/ONERAdrag). To load the data, just replace the data_dir variable with the appropriate path to the dataset.

In [2]:
data_dir = '../data/ONERAdrag'
X = np.load(data_dir+'/X.npy')
Y = np.load(data_dir+'/Y.npy')
DY = np.load(data_dir+'/DY.npy')

Designate the number of (randomly chosen) data points to train with.

In [3]:
num_train = 100

In [4]:
ind = list(range(X.shape[0]))
np.random.shuffle(ind)
ind_train = ind[:num_train]

In [5]:
X_train = X[ind_train,:]
Y_train = Y[ind_train]
DY_train = DY[ind_train,:]

To train the NN model, we first learn the $k$ dimensional linear subspace $U$ via active subspaces--see [Bollinger (2022)](???) (link will be available once published). If given datapoint is labeled $x$, then reduced data is given by $U^T x$, labeled "UTX" in this code.

In [6]:
k = 3

ss = rom.subspaces
U = ss.AS(DY_train,k)

Then, we train the NN surrogate model. For this example, our hyperparameter choice will be:
- h = hidden dimension size = 256
- alpha = regularizing hyperparameter = 1e-7
- num_outer = number of outer iterations for alternating minimization scheme = 3
- num_epoch = number of inner iterations for NN training = 200

In [7]:
h = 256

In [8]:
model = rom.surr_model.NN_alt(U=U,dim_layers=[k,h,1],alpha=1e-7)
model.train([X_train, Y_train],dataset_val=[X,Y],num_outer=3,num_epoch=200)

Training parameters:
	Number of outer iterations = 3
	Number of inner iterations for NN = 200
	Batch size for training NN = 16

Outer iteration: 1

Epoch 0/200 | Train Rel Loss 4.033078 | Validation Rel Loss: 4.11792803
Epoch 100/200 | Train Rel Loss 0.070650 | Validation Rel Loss: 0.09425871
Epoch 200/200 | Train Rel Loss 0.078292 | Validation Rel Loss: 0.09263412

Outer iteration: 2

Epoch 0/200 | Train Rel Loss 0.019185 | Validation Rel Loss: 0.09483729
Epoch 100/200 | Train Rel Loss 0.036024 | Validation Rel Loss: 0.10201846
Epoch 200/200 | Train Rel Loss 0.017165 | Validation Rel Loss: 0.09367252

Outer iteration: 3

Epoch 0/200 | Train Rel Loss 0.012191 | Validation Rel Loss: 0.09158520
Epoch 100/200 | Train Rel Loss 0.011806 | Validation Rel Loss: 0.08979782
Epoch 200/200 | Train Rel Loss 0.014197 | Validation Rel Loss: 0.09096321


Using the trained model, we predict output over the entire dataset and report error.

In [9]:
Y_calc = model.predict(X)

print(f'relative error = {rom.utils.rel_error(Y,Y_calc)}')

relative error = 0.09096320760276319
